## Lets check the customer data and create dim_customer table in gold layer

In [0]:
%sql

SELECT customer_key,
       customer_id,
       customer_name,
       email,
       phone,
       country,
       state,
       city
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY customer_id) AS rn,
           customer_id,
           customer_name,
           email,
           phone,
           country,
           state,
           city
    FROM salesdb.silver.crm_sales_data
    WHERE customer_id IS NOT NULL
) t
WHERE rn = 1


In [0]:
query =  """
SELECT customer_key,
       customer_id,
       customer_name,
       email,
       phone,
       country,
       state,
       city
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY customer_id) AS rn,
           customer_id,
           customer_name,
           email,
           phone,
           country,
           state,
           city
    FROM salesdb.silver.crm_sales_data
    WHERE customer_id IS NOT NULL
) t
WHERE rn = 1
 """
df_customer = spark.sql(query)
display(df_customer)
df_customer.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_customer")

In [0]:
%sql

select * from salesdb.silver.crm_sales_data

In [0]:
%sql

select salesrep_key, rn,sales_rep_id, sales_rep_name from 
(
SELECT ROW_NUMBER() OVER (ORDER BY sales_rep_id) AS salesrep_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY sales_rep_id ORDER BY sales_rep_id) AS rn,
           sales_rep_id,
           sales_rep_name
    FROM salesdb.silver.crm_sales_data 
    WHERE sales_rep_id IS NOT NULL
) as t where rn=1



In [0]:
query =  """
SELECT ROW_NUMBER() OVER (ORDER BY sales_rep_id) AS salesrep_key,  -- Surrogate key
       sales_rep_id,
       sales_rep_name
FROM (
    SELECT 
           ROW_NUMBER() OVER (PARTITION BY sales_rep_id ORDER BY sales_rep_id) AS rn,
           sales_rep_id,
           sales_rep_name
    FROM salesdb.silver.crm_sales_data 
    WHERE sales_rep_id IS NOT NULL
) t
WHERE rn = 1
 """
df_salesrep_ = spark.sql(query)
display(df_salesrep_)
df_salesrep_.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_salesrep")



In [0]:
%sql
SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
      rn
       product_id,
       product_name,
       category
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY product_id) AS rn,
           product_id,
           product_name,
           category
    FROM salesdb.silver.crm_sales_data 
    WHERE product_id IS NOT NULL
) t WHERE rn = 1


In [0]:
query =  """
SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
       product_id,
       product_name,
       category
FROM (
    SELECT ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY product_id) AS rn,
           product_id,
           product_name,
           category
    FROM salesdb.silver.crm_sales_data 
    WHERE product_id IS NOT NULL
) t
WHERE rn = 1
 """


df_product = spark.sql(query)
display(df_product)
df_product.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_product")